In [ ]:
import os,sys
b_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'model'))
sys.path.insert(0, b_directory)
from pan_epitope_double import *
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

In [ ]:
class sx_Dataset(Dataset):
    def __init__(self,data1,data2,data3,data4):
        self.x1 = data1
        self.x2 = data2
        self.x3 = data3
        self.x4 = data4
        self.len = data1.shape[0]
 
    def __getitem__(self, index):
        return self.x1[index],self.x2[index],self.x3[index],self.x4[index]
 
    def __len__(self):
        return self.len

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
def cal_(beta_train_emb,alpha_train_emb,ep_train_emb,train_labels,seed=1,lr=0.0001,EPOCH=100,BATCH_SIZE=16, device='cuda:1'):
    
    set_seed(seed)
    dataset = sx_Dataset(beta_train_emb,alpha_train_emb,ep_train_emb,train_labels)
    train_dataloader=DataLoader(dataset=dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4,drop_last=True)

    model=classification_model(tcr_dim=beta_train_emb.shape[-1], pep_dim=ep_train_emb.shape[-1], 
                               )
    model=model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(EPOCH):
        model.train()       

        for tra_step, (btr,atr,pep,tl) in enumerate(train_dataloader):   
            pep=torch.tensor(pep,dtype=torch.float32).to(device) 
            btr=torch.tensor(btr,dtype=torch.float32).to(device)    
            atr=torch.tensor(atr,dtype=torch.float32).to(device) 
            tl=torch.tensor(tl,dtype=torch.float32).to(device)

            pred = model(btr,atr,pep).flatten()  
            loss = F.binary_cross_entropy(pred,tl)

            optimizer.zero_grad()
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
            
        torch.save(model,'./model_'+str(epoch)+'.pt')

In [ ]:
beta_train_emb=np.load('./beta_train_emb.npy')
alpha_train_emb=np.load('./alpha_train_emb.npy')
ep_train_emb=np.load('./ep_train_emb.npy')
train_labels=np.load('./train_labels.npy')

In [ ]:
cal_(beta_train_emb,ep_train_emb,train_health_tcr_emb)